Initialization

In [73]:
from scipy.optimize import minimize
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp,Operator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit.quantum_info import Statevector,random_statevector
import math
import json
import os
import random

GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
MAGENTA = "\033[95m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
STANDARD = "\033[0m"

%run config.py

Load the directories and identify the project inside them.

In [74]:
def rotfitness(parameters, *args):
	qbits=int(len(parameters)/3)

	qc = QuantumCircuit(qbits)
	for i in range(qbits):
		qc.rx(float(parameters[3*i]), i)
		qc.ry(float(parameters[3*i+1]), i)
		qc.rz(float(parameters[3*i+2]), i)
	
	# print (qc.draw())
	qc=qc.reverse_bits()

	complexArray = []
	complexArray.append(complex(1,0))
	for i in range(2**qbits-1):
		complexArray.append(complex(0,0))

	sv = Statevector(complexArray)
	sv = sv.evolve(qc)

	outlistcomplex = sv.data.tolist()
	# print(outlistcomplex)
	# print (args)

	sigma = 0.0
	for i in range(len(outlistcomplex)):
		sigma  += abs( args[i].real - outlistcomplex[i].real)
		sigma  += abs( args[i].imag - outlistcomplex[i].imag)

	sigma = sigma/len(outlistcomplex)
	# print (sigma)
	return sigma

In [75]:
def guessrotations(target):
	# target is a statevector
	length = len(target)
	qbits = int(math.log2(length))
	# print (qbits)
	initial_guess = [1.0,] * 3*qbits
	# print(initial_guess)
	result = minimize(rotfitness, initial_guess, args=tuple(target))
	return result.x.tolist()

In [76]:
# List all the directories, only the directories
dirs = [d for d in os.listdir('.') if os.path.isdir(d)]
activeproj = {}
base = ""
for d in dirs:
	if os.path.exists(d+"/active"):
		with open(d+"/active","r") as f:
			ptype = f.read().strip()
			activeproj[d] = ptype
		if os.path.exists(d+"/base"):
			base = d
			if ptype != "qiskit":
				print("Warning: base project is not of type qiskit")
		
# print("Active projects: ",activeproj)
# print("Static data: ",staticdata)
# print("Base project: ",base)

In [77]:
def getRandomStateVectorWhole(qbits):
	sv=random_statevector(2**qbits)
	rot=guessrotations(sv)
	return sv,rot

In [78]:
def getRandomStateVectorRotations(qbits):
	states=[]
	rots=[]
	for i in range(qbits):
		# Random rotation 0 4pi
		theta = 2*math.pi*random.random()
		phi = 2 *math.pi*random.random()

		rot=[theta,phi,0]

		qc = QuantumCircuit(1)
		qc.rx(theta, 0)
		qc.ry(phi, 0)
	
		complexArray = []
		complexArray.append(complex(1,0))
		complexArray.append(complex(0,0))

		sv = Statevector(complexArray)
		sv = sv.evolve(qc)

		states.append(sv)
		rots.append(rot)

	rot=[]
	for i in range(qbits):
		rot+=rots[i]

	sv=states[0]
	for i in range(1,qbits):
		newsv=[]
		for j in range(len(sv.data)):
			for k in range(2):
				newsv.append(sv.data[j]*states[i].data[k])
		sv=Statevector(newsv)

	print(sv)
	return sv,rot

In [79]:
def getRandomStateVectorSingle(qbits):
	states=[]
	rots=[]
	for i in range(qbits):
		sv=random_statevector(2**qbits)
		# sv= Statevector([1/math.sqrt(2), 1j/math.sqrt(2)])
		# sv= Statevector([0, 1])
		states.append(sv)
		print ("State: ",sv)
		ai = sv.data[0].imag
		ar = sv.data[0].real
		bi = sv.data[1].imag
		br = sv.data[1].real
		d2 = br**2+ai**2
		print ("d2: ",d2)
		c2 = ai**2/(d2)
		print ("c2: ",c2)
		phi = 2.0*math.asin(math.sqrt(d2))
		print ("phi: ",phi)
		theta = 2.0* math.asin(math.sqrt(c2))
		print ("theta: ",theta)
		print ("rx:")
		print ( math.cos(theta/2), "-i",math.sin(theta/2))
		print ( "-i", math.sin(theta/2), math.cos(theta/2))

		print ("ry:")
		print ( math.cos(phi/2), "-",math.sin(phi/2))
		print ( math.sin(phi/2), math.cos(phi/2))

		g0ar = math.cos(theta/2)
		g0bi = math.sin(theta/2)
		print ("g0 = rx * 0")
		print (g0ar, "-i",g0bi)

		print ("g1 = ry * g0")
		

		rot = [theta,phi,0]
	# sv=random_statevector(2**qbits)
	# rot=guessrotations(sv)
	return sv,rot

# getRandomStateVector(2)

Create a set of inputs for the test cases. They should be normalized values, let's use the qiskit random_statevector function to generate them.

In [80]:
if fundamental:
	inputs = []
	rots = []
	outlist=[[1.0,0.0]]
	rotlist=[0.0,] * 3 * qbits
	for i in range(2**qbits-1):
		outlist.append([0.0,0.0])
	inputs.append(outlist)
	rots.append(rotlist)

	with open('inputs.json', 'w') as f:
		json.dump(inputs, f)

	with open('rots.json', 'w') as f:
		json.dump(rots, f)
else:
	if staticdata:
		with open('inputs.json', 'r') as f:
			inputs = json.load(f)
		with open('rots.json', 'r') as f:
			rots = json.load(f)
	else:
		inputs = []
		rots = []
		for i in range(numiter):
			sv,rot=getRandomStateVectorRotations(qbits)
			outlist=[]
			for i in range(len(sv)):
				outlist.append([sv[i].real,sv[i].imag])
			rots.append(rot)
			inputs.append(outlist)


		with open('inputs.json', 'w') as f:
			json.dump(inputs, f)

		with open('rots.json', 'w') as f:
			json.dump(rots, f)

Statevector([-0.37185829+0.08539768j, -0.17274628-0.39341755j,
             -0.3553265 -0.41114864j,  0.3800698 -0.47964239j],
            dims=(2, 2))
Statevector([-0.32216856+0.56096761j, -0.3333142 -0.33091266j,
             -0.28865852-0.39116779j,  0.32840293-0.12937059j],
            dims=(2, 2))
Statevector([-0.01285871+0.20695836j,  0.10329228+0.24140156j,
             -0.56390734-0.15203976j, -0.72437427+0.14910806j],
            dims=(2, 2))
Statevector([-0.13953253+0.1152325j ,  0.45863982-0.38851027j,
              0.21006619+0.07894676j, -0.70096235-0.2534814j ],
            dims=(2, 2))
Statevector([-0.17321304-0.7688715j , -0.0335654 -0.22209047j,
             -0.47820448+0.27392682j, -0.14151821+0.06811913j],
            dims=(2, 2))
Statevector([-0.73490186-0.09329438j,  0.18393908-0.35864002j,
              0.46938607+0.04979659j, -0.11247442+0.23088025j],
            dims=(2, 2))
Statevector([-0.42551317-0.00259754j, -0.11532468+0.66069905j,
              0.0928748 -

Convert the inputs to the format expected by the single project.

In [81]:
for p in activeproj:
	if activeproj[p] == "qiskit" or activeproj[p] == "pennylane" or activeproj[p] == "spinqsim" or activeproj[p] == "spinq":
		inputsJson = []
		for i in inputs:
			inputJson=[]
			for j in i:
				val= [j[0], j[1]]
				inputJson.append(val)
			inputsJson.append( inputJson )

		with open(p+'/inputs.json', 'w') as f:
			json.dump(inputsJson, f)
		with open(p+'/rots.json', 'w') as f:
			json.dump(rots, f)

	if activeproj[p] == "bmqsim":
		inputsJson = []
		for i in inputs:
			inputJson=[]
			for j in i:
				val= {"Real": j[0], "Imag": j[1]}
				inputJson.append(val)
			inputsJson.append({ "Vector": inputJson })

		with open(p+'/inputs.json', 'w') as f:
			json.dump(inputsJson, f)
		with open(p+'/rots.json', 'w') as f:
			json.dump(rots, f)

	if activeproj[p] == "bmqsim_hw" or activeproj[p] == "quest":
		inputValues=[]
		for i in inputs:
			for j in i:
				inputValues.append(j[0])
				inputValues.append(j[1])

		with open(p+'/inputs.txt', 'w') as f:
			for item in inputValues:
				f.write("%s\n" % item)
		with open(p+'/rots.json', 'w') as f:
			json.dump(rots, f)
			

Run all the projects

In [82]:
for p in activeproj:	
	cwd = os.getcwd()
	os.chdir(p)
	if os.path.exists("circuit.py"):
		if activeproj[p] == "spinqsim" or activeproj[p] == "spinq":
			! source /tools/Conda/etc/profile.d/conda.sh && conda activate spinq && python circuit.py && conda deactivate
		elif activeproj[p] == "pennylane":
			! source /tools/Conda/etc/profile.d/conda.sh && conda activate pennylane && python circuit.py && conda deactivate
		else:
			%run circuit.py
	elif os.path.exists("circuit.ipynb"):
		%run circuit.ipynb
	os.chdir(cwd)

[{"Vector": [{"Real": -0.3718582919675802, "Imag": 0.08539768106431167}, {"Real": -0.17274627502621545, "Imag": -0.3934175527084198}, {"Real": -0.35532650251415565, "Imag": -0.4111486370140053}, {"Real": 0.38006980466906753, "Imag": -0.4796423864929761}]}, {"Vector": [{"Real": -0.32216856031866203, "Imag": 0.5609676100360086}, {"Real": -0.3333142022929231, "Imag": -0.3309126641634325}, {"Real": -0.2886585233864885, "Imag": -0.39116778664599783}, {"Real": 0.3284029253751829, "Imag": -0.12937058649710884}]}, {"Vector": [{"Real": -0.012858706983379597, "Imag": 0.2069583597501194}, {"Real": 0.10329228430997861, "Imag": 0.24140156210554373}, {"Real": -0.5639073446998121, "Imag": -0.1520397602980943}, {"Real": -0.7243742723095188, "Imag": 0.14910805577649455}]}, {"Vector": [{"Real": -0.13953253030464433, "Imag": 0.11523249994439028}, {"Real": 0.4586398174539466, "Imag": -0.38851026762493196}, {"Real": 0.21006618953719702, "Imag": 0.07894676007687793}, {"Real": -0.7009623535140007, "Imag": -0

OUTPUT checks the output of the projects and compares them with the pivot project.


In [83]:
passed = True
passwdProjects = {}
for p in activeproj:
	if (activeproj[p] == "qiskit" and p!=base) or activeproj[p] == "spinqsim" or activeproj[p] == "pennylane":
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.json', 'r') as f:
			outputs = json.load(f)

		if len(qiskitoutputs) != len(outputs):
			print("Warning: Different number of outputs")

		qvars = []
		
		probs = []

		for i in range(len(qiskitoutputs[0])):
			qvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			spinqoutput = outputs[i]
			prob = []
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				qreal2 = float(spinqoutput[j][0])
				qimag2 = float(spinqoutput[j][1])
				if useprob:
					qvars[j][0]=pow(pow(qreal,2)+pow(qimag,2)-pow(qreal2,2)-pow(qimag2,2),2)
				else:
					qvars[j][0] += pow(qreal - qreal2, 2)
					qvars[j][1] += pow(qimag - qimag2, 2)

				prob.append(pow(qreal,2)+pow(qimag,2))
			probs.append(prob)

		with open(p+'/probs.json', 'w') as f:
			json.dump(probs, f)

		for i in range(len(qvars)):
			qvars[i][0] = pow(qvars[i][0]/len(qiskitoutputs), 0.5)
			qvars[i][1] = pow(qvars[i][1]/len(qiskitoutputs), 0.5)
			if qvars[i][0] > passthreshold or qvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(qvars, indent=4))
		
		if p not in passwdProjects:
			passwdProjects[p] = True

	if activeproj[p] == "spinq":
		if not useprob:
			panic("spinq only supports useprob mode")

		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/probs.json', 'r') as f:
			probs = json.load(f)

		if len(qiskitoutputs) != len(probs):
			print("Warning: Different number of outputs")

		qvars = []

		for i in range(len(qiskitoutputs[0])):
			qvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			spinqoutput = probs[i]
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				prob = float(spinqoutput[j])
				if useprob:
					qvars[j][0]=pow(pow(qreal,2)+pow(qimag,2)-prob,2)

		for i in range(len(qvars)):
			qvars[i][0] = pow(qvars[i][0]/len(qiskitoutputs), 0.5)
			qvars[i][1] = pow(qvars[i][1]/len(qiskitoutputs), 0.5)
			if qvars[i][0] > passthreshold or qvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(qvars, indent=4))

		if p not in passwdProjects:
			passwdProjects[p] = True
			
	if activeproj[p] == "bmqsim":
		
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.json', 'r') as f:
			bmqsimoutputs = json.load(f)

		if len(qiskitoutputs) != len(bmqsimoutputs):
			print("Warning: Different number of outputs")

		bmvars = []
		probs = []
		for i in range(len(qiskitoutputs[0])):
			bmvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			bmqsimoutput = bmqsimoutputs[i]["Vector"]
			prob = []
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				breal = float(bmqsimoutput[j]["Real"])
				bimag = float(bmqsimoutput[j]["Imag"])
				if useprob:
					bmvars[j][0]=pow(pow(qreal,2)+pow(qimag,2)-pow(breal,2)-pow(bimag,2),2)
				else:
					bmvars[j][0] += pow(qreal - breal, 2)
					bmvars[j][1] += pow(qimag - bimag, 2)

				prob.append(pow(breal,2)+pow(bimag,2))
			probs.append(prob)

		with open(p+'/probs.json', 'w') as f:
			json.dump(probs, f)

		for i in range(len(bmvars)):
			bmvars[i][0] = pow(bmvars[i][0]/len(qiskitoutputs), 0.5)
			bmvars[i][1] = pow(bmvars[i][1]/len(qiskitoutputs), 0.5)
			if bmvars[i][0] > passthreshold or bmvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(bmvars, indent=4))

		if p not in passwdProjects:
			passwdProjects[p] = True

	if activeproj[p] == "bmqsim_hw" or activeproj[p] == "quest":
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.txt', 'r') as f:
			bmqsimoutputs = []
			for k in range(len(qiskitoutputs)):
				bmqsimoutput=[]
				for i in range(2 ** qbits):
					line = f.readline()
					vreal=float(line)
					line = f.readline()
					vimm=float(line)
					bmqsimoutput.append([vreal,vimm])
				bmqsimoutputs.append(bmqsimoutput)

		if len(qiskitoutputs) != len(bmqsimoutputs):
			print("Warning: Different number of outputs")

		bmvars = []
		probs = []

		for i in range(len(qiskitoutputs[0])):
			bmvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			bmqsimoutput = bmqsimoutputs[i]
			prob = []
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				breal = float(bmqsimoutput[j][0])
				bimag = float(bmqsimoutput[j][1])
				if useprob:
					bmvars[j][0]=pow(pow(qreal,2)+pow(qimag,2)-pow(breal,2)-pow(bimag,2),2)
				else:
					bmvars[j][0] += pow(qreal - breal, 2)
					bmvars[j][1] += pow(qimag - bimag, 2)
				prob.append(pow(breal,2)+pow(bimag,2))
			probs.append(prob)

		for i in range(len(bmvars)):
			bmvars[i][0] = pow(bmvars[i][0]/len(qiskitoutputs), 0.5)
			bmvars[i][1] = pow(bmvars[i][1]/len(qiskitoutputs), 0.5)
			if bmvars[i][0] > passthreshold or bmvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False
		with open(p+'/probs.json', 'w') as f:
			json.dump(probs, f)
			
		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(bmvars, indent=4))

		if p not in passwdProjects:
			passwdProjects[p] = True

Average error on bmqsim:
[
    [
        1.50988932670788e-09,
        0.0
    ],
    [
        8.447322190863445e-10,
        0.0
    ],
    [
        1.578461945122163e-08,
        0.0
    ],
    [
        2.6515666400533945e-08,
        0.0
    ]
]
Average error on spinqq:
[
    [
        0.004073196960353991,
        0.0
    ],
    [
        0.0032800442908969257,
        0.0
    ],
    [
        0.023650240211082443,
        0.0
    ],
    [
        0.02444339288053945,
        0.0
    ]
]
Average error on bmqsim_hw:
[
    [
        1.4561498260445125e-09,
        0.0
    ],
    [
        8.122895974422628e-10,
        0.0
    ],
    [
        1.4743810891644555e-08,
        0.0
    ],
    [
        2.028810716838709e-08,
        0.0
    ]
]
Average error on pennylane:
[
    [
        8.228515941976643e-19,
        0.0
    ],
    [
        2.7428386473255477e-19,
        0.0
    ],
    [
        3.510833468576701e-17,
        0.0
    ],
    [
        1.0971354589302191e-17,
      

In [84]:
if not quiet:
	if passed:
		print(f"{CYAN}Overall{STANDARD}: {GREEN}Passed{STANDARD}")
	else:
		print(f"Overall: {RED}Failed{STANDARD}")
	print ("")
	print (f"{CYAN}Detailed results{STANDARD}:")
	for p in passwdProjects:
		if passwdProjects[p]:
			print(f"\t{MAGENTA}"+p+f"{STANDARD}"+f": {GREEN}Passed{STANDARD}")
		else:
			print(f"\t{MAGENTA}"+p+f"{STANDARD}"+f": {RED}Failed{STANDARD}")

Overall: Passed

Detailed results:
	bmqsim: Passed
	spinqq: Passed
	bmqsim_hw: Passed
	pennylane: Passed
	quest: Passed
	spinq: Passed
	bmqsim_alveo: Passed
	qiskitrot: Passed
